In [1]:
import os
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import StorageContext, load_index_from_storage

cache_dir="E:\Cache\Hugging_Face"

project_path = os.path.abspath(os.path.relpath('../../../', os.getcwd()))
data_path = os.path.join(project_path, 'FT4LLM/Data')
knowledge_path=os.path.join(data_path, 'articles')
prompt_path = os.path.join(data_path, 'prompt')

In [2]:
Settings.embed_model = HuggingFaceEmbedding(model_name="ekorman-strive/bge-large-en-v1.5", cache_folder=cache_dir)
Settings.llm = None
Settings.chunk_size = 128
Settings.chunk_overlap = 25
documents = SimpleDirectoryReader(knowledge_path).load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir=cache_dir+'/vector_cache')

E:\anaconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
E:\anaconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM is explicitly disabled. Using MockLLM.


In [3]:

storage_context = StorageContext.from_defaults(persist_dir=cache_dir + '/vector_cache')
index = load_index_from_storage(storage_context)
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)
